# form

> this builds up an `.md` with a lot of nastiness to get to an HTML form to propose a new config file

In [ ]:
import re
from pathlib import Path

import importnb
import IPython
import jinja2

In [ ]:
URL = "https://github.com/deathbeds/jupyak/new/main"

In [ ]:
with importnb.Notebook():
    from jupyak.tasks import load_tasks
    from jupyak.tasks._yak import Yak

In [ ]:
def load_yak():
    load_tasks()
    return Yak({"repos": {}})

In [ ]:
REPOS_TMPL = r"""
<fieldset>
<legend><h2>customize GitHub checkout</h2></legend>

<blockquote>
All of the repositories below will be checked out at the current <code>main</code> branch as a <em>baseline</em>.
Check one or more repos to provide use a differnt branch, PR, or tag, or provide one or more PRs to
<i>merge with</i> it, and/or customize the <a href="https://git-scm.com/docs/merge-strategies" target="blank">merge strategy and options</a> to work around merge conflicts.
</blockquote>

{% for name in yak.repos %}
    <input type="checkbox" name="show-repo-{{ name }}" id="show-repo-{{ name }}"/>
    <label class="show-repo-label" for="show-repo-{{ name }}">{{name}}</label>
{% endfor %}
<table style="position: relative;">
    <thead style="position: sticky; top: 0;">
        <tr>
            <th>repo</th>
            <th>baseline</th>
            <th>merge with</th>
            <th>merge strategy</th>
            <th>merge options</th>
        </tr>
        <tr>
            <td><blockquote>
                GitHub repository to check out and build
            </blockquote></td>
            <td><blockquote>
                starting point GitHub reference
            </blockquote></td>
            <td><blockquote>
                optional space-delimted list of references to merge into the baseline
            </blockquote></td>
            <td><blockquote>
                merge strategy
            </blockquote></td>
            <td><blockquote>
                additional space-delimeted <code>-X</code> options to pass to <code>git merge</code>
            </blockquote></td>
        </tr>
    </thead>
    <tbody>
        {% for name, repo in yak.repos.items() %}
        {% set stem = "repos|" ~ name ~ "|github" %}
        {% set id_stem = "repos-" ~ name ~ "-github" %}
        {% set gh = repo.github %}
        {% set gh_pattern = "(tree/[^s]+|pull/\d+|releases/tag/[^s]+)" %}
        <tr class="repo" id="repo-{{ name }}">
            <th><code>{{ gh.url }}/</code></th>
            <td>
                <input id="{{ id_stem }}-baseline"
                    name="{{ stem }}|baseline"
                    type="text"
                    title="the baseline GitHub URL for {{ name }}"
                    spellcheck="false"
                    placeholder="{{ gh.baseline }}"
                    pattern="^$|^{{ gh_pattern }}"
                />
                <label for="{{ id_stem }}-baseline">
                    must be empty, or one of:<br/>
                    <code>pull/{:number}</code><br/>
                    <code>tree/{:branch}</code><br/>
                    <code>releases/tag/{:tag}</code>
                </label>
            </td>
            <td>
                <input id="{{ id_stem }}-merge_with"
                    name="{{ stem }}|merge_with"
                    title="one or more space-delimited GitHub URLs to merge into the {{ name }} baseline"
                    type="text"
                    spellcheck="false"
                    placeholder="pull/{:number} tree/{:branch} releases/tag/{:tag}"
                    pattern="^$|^{{ gh_pattern }}(\s+{{ gh_pattern}})*"
                />
                <label for="{{ id_stem }}-merge_with">
                    must be empty, or one or more (separated by space) of:<br/>
                    <code>pull/{:number}</code><br/>
                    <code>tree/{:branch}</code><br/>
                    <code>releases/tag/{:tag}</code>
                </label>
            </td>
            <td>
                <select
                    name="{{ stem }}|merge_strategy"
                    title="choose a different git merge strategy"
                >
                    <option value="">ort (default)</option>
                    <option>resolve</option>
                    <option>octopus</option>
                    <option>ours</option>
                    <option>subtree</option>
                </select>
            </td>
            <td>
                <input
                    name="{{ stem }}|merge_options"
                    type="text"
                    title="add space-delimted -X options for the merge strategy"
                />
            </td>
        </tr>
        {% endfor %}
    </tbody>
</table>
"""

In [ ]:
def make_repos_form(yak: Yak):
    return [jinja2.Template(REPOS_TMPL).render(yak=yak)]

In [ ]:
REPO_STYLE_TEMPL = """
{% for name in yak.repos %}
#show-repo-{{ name }}:not(:checked) ~ table tbody #repo-{{ name }} {
    display: none;
}
{% endfor %}
"""

In [ ]:
def make_repos_style(yak: Yak):
    return [jinja2.Template(REPO_STYLE_TEMPL).render(yak=yak)]

In [ ]:
LITE_TMPL = r"""
<fieldset>
<legend>customize JupyterLite</legend>

<blockquote>
If given, an optional <a target="_blank" href="https://gist.github.com/">gist</a> will be cloned
to provide the content of the JupyterLite site.
If the gist contains <code>jupyter_lite_config.json</code> and/or <code>jupyter-lite.json</code>,
this will be merged into the generated
<a target="_blank" href="https://jupyterlite.readthedocs.io/en/latest/howto/configure/config_files.html">configuration</a>
of <code>jupyter lite build</code> and the runtime application.
</blockquote>

<input id="lite-gist"
     type="text"
     name="lite|gist"
     pattern="^$|^[a-z\d]{20,}$"
     placeholder="gist ID"
     />
<label for="lite-gist">the gist ID must be empty, or contain 20+ letters and numbers</label>
</fieldset>
"""

In [ ]:
def make_lite_form(yak: Yak):
    return [jinja2.Template(LITE_TMPL).render(yak=yak)]

In [ ]:
def make_form():
    yak = load_yak()
    chunks = []
    style_chunks = []
    script_chunks = []
    for trait_name in ["repos", "lite"]:
        value = yak.trait_values()[trait_name]
        maker = globals().get(f"make_{trait_name}_form")
        if maker:
            chunks += maker(yak)
        styler = globals().get(f"make_{trait_name}_style")
        if styler:
            style_chunks += styler(yak)

    return [
        "<style>",
        *style_chunks,
        "</style>",
        """<form id="new">""",
        *chunks,
        "</form>",
        *script_chunks,
    ]

In [ ]:
def write_form(dest: Path, **options):
    dest.parent.mkdir(parents=True, exist_ok=True)
    final_tmpl = dest.parent / "_templates/new-form.html"
    chunks = [
        "# request a preview site",
        "",
        "> Make selections below for the most common configuration options, then use the ",
        "> _start pull request_ form to be redirected to GitHub.",
        "> A large number of other configuration options are _possible_, but are not yet fully described.",
        "",
        *make_form(),
        final_tmpl.read_text(encoding="utf-8"),
    ]
    txt = re.sub(r"^ +", "", "\n".join(chunks), flags=re.M)
    dest.write_text(txt, encoding="utf-8")

In [ ]:
if __name__ == "__main__":
    IPython.display.display(IPython.display.HTML("\n".join(make_form())))